# Experimenting with LLM

In [25]:
from openai import OpenAI
import os # To manoeuver around files
from dotenv import load_dotenv # To loads files from .env
from urllib.parse import quote # To handle user input for URL

# Sadly, following the LLM course is a lil out of date now, so the below won't make much sense...
# from langchain.tools import WebBaseLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings

# Let's instead go for agentic stuff directly
from langchain.agents import create_agent
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from dataclasses import dataclass
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


load_dotenv() # Actually loading files from .env

True

## Following the usual LLM course...

In [17]:
# Assign API Key from .env to a variable usable here
openai_API_key = os.getenv("LLM_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("LLM_API_KEY")

# Prints first 10 chars for loading confirmation
print(openai_API_key[:10])
print(OPENAI_API_KEY[:10])

sk-proj-8T
sk-proj-8T


In [8]:
# Assign API Key
client = OpenAI(
  api_key=openai_API_key
)


#Create a basic system prompt - TODO must import this from a separate file to modularise later.
system_prompt = "You are a vegan nutritionist. Focus on plantbased foods."


# Creating the function
def generate_text(prompt:str) -> str:
    """Used to get the main model reply"""
    # Make API call
    response = client.responses.create(
        model="gpt-5-nano",
        input=[
                {
                    "role": "developer",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        store=True,
    )
    # Prints out answer
    print(response.output_text)

# generate_text("Give me a 3-word sentence about eating meat.")


## LangChain Documentation
### Creating an Agent
#### System Prompt

In [ ]:
# TODO: Change accordingly... 
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. 
If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

#### Tools

In [19]:
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

#### Loading Model

In [18]:

model = init_chat_model(
    "gpt-4.1",
    temperature=0.5,
    timeout=10,
    max_tokens=235
)

#### Defining Response Format

In [22]:
# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

#### Adding Memory

In [21]:
checkpointer = InMemorySaver()

#### Testing a Run

In [27]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like the sun is on a roll in Florida—it's always sunny-side up! Don't forget your shades, unless you want to be egg-stra squinty.", weather_conditions="It's always sunny in Florida!")
ResponseFormat(punny_response="No prob-llama! If you need more weather wisdom, just give me a shout—I'm always ready to rain down some forecasts!", weather_conditions=None)
